Working on the Post-training quantization of the model and the subset of the dataset. 
* Working on the model already trained with the CMS dataset.
* The model has been quantized using the `tf.lite.converter`
* Further, we are working with sample of the dataset(100 for our case). 
* Quantizing the sample of dataset using INT8 quantization


As written on here, we were working on the stuff with the debugging of the model and working out the task. 
* Perform the quantization of the model
* Convert all of the data 
* Put all stuff in a notebook along with the model interface.

## `Task`
Standardize the P_T value before quantizing.

Three P_T plots:

Raw P_T (Number of particles vs P_T (GeV))
P_T after standardization (X-axis should be arbitrary units)
After int8 quantization. Centers at 0 and range should be between -127, 127)
Upon standardization, there could be a few values which could be out of the range of (-127, 127). How to deal with the outliers? One of the methods is to put all of them in the last bin. Are there any other methods available?

Fix bin size does not give resolution. We can lose information if we discard those outliers.

Print the true and predicted P_T values. Should be in the INT8 range.

Put on the distribution if it makes sense.

We need to standradization to whole datasample which is 100 in our case.
* Plot the datasample before and after standradization ??
* Or do we only need to standradize the True P_T?
* What type of output you have right now?

`The dataset contains input features and target features consist of different things like pT, eta, phi etc. so all of those need to be standardized and quantized separately. but you can start with just pT` Task:

1. Since working only on the pT, quntize it after standradization. Further Check the plots before and after quantization.

Quantize the dataset after stnadradization and then do the inference. Check the output plots

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ROOT

2023-12-26 15:44:29.290847: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-26 15:44:36.408911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/03


In [3]:
import sys
sys.path += ["../../../MLPF/mlpf/particleflow/mlpf/"]
from tfmodel.model_setup import make_model
from tfmodel.utils import parse_config


In [4]:
config, _ = parse_config("../../../MLPF/mlpf/particleflow/parameters/clic.yaml") #positions on the lxplus


In [5]:

model = make_model(config, tf.float32)
model.build((1, None, config["dataset"]["num_input_features"]))

2023-12-26 15:49:08.261369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 15:49:09.215487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 15:49:09.220625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
model.summary()

Model: "pf_net_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 node_encoding (Sequential)  (1, None, 256)            70912     
                                                                 
 input_encoding_clic (Input  multiple                  0         
 EncodingCLIC)                                                   
                                                                 
 cg_id_0 (CombinedGraphLaye  multiple                  440128    
 r)                                                              
                                                                 
 cg_id_1 (CombinedGraphLaye  multiple                  440128    
 r)                                                              
                                                                 
 cg_id_2 (CombinedGraphLaye  multiple                  440128    
 r)                                                   

In [10]:
model.load_weights("../../../MLPF/mlpf/weights-96-5.346523.hdf5", skip_mismatch=False, by_name=True)
## These files hosted at https://huggingface.co/jpata/particleflow/tree/clic_clusters_v1.6

OSError: Unable to open file (file signature not found)